# MathEngine Tutorial

### Enter device and channel info

Enter your informationg and press ctrl+enter to run a block of code

In [ ]:
# Libraries for manipulating the data
import numpy as np
import matplotlib.pyplot as plt

# Data source
deviceSerial = 'yourSerial'
inSensor     = 'yourSensorName'
inChannel    = 'yourChannelName'
outChannel   = inSensor + '_scaled'

# Uncomment the next two lines to specify start and end time for the data you want
#start = timestampInSeconds *  NANO_PER_SEC
#end = timestampInSeconds * NANO_PER_SEC

### Get the data from SensorCloud

In [ ]:
repo = TimeSeriesRepo(deviceSerial)

- Get a list of all data series on a channel, these series are seperated by sample rate

In [ ]:
inSeries  = repo.getAllTimeSeries(inSensor, inChannel)
print len(inSeries), "series found"

### Strip out the data

Each data series is a virtual list of tuples. The tuples have the form (timestamp, value)

In [ ]:
series = inSeries[0]
values = series.getData()
timestamps= series.getTimeStamps()

print timestamps[0]
print values[0]

After manipulating the data you can zip it with the timestamps to restore the list of tuples

In [ ]:
# We cast values to a numpy array so we can perform math operations across all of the values
# Python will treat the numpy array as it would a standard list
array = np.array(values)

# Scale the values by 10
array = array*10
print "Scaled %s points..." % len(array)

# Zip them back up with the timestamps
outData = zip(timestamps, array)

Now we have the scaled data in format thats ready to be uploaded back to SensorCloud

### Alternate Data Retrieval
Rather than load all of the data into memory at once and then process it, we can iterate over the series.  Under the hood, the TimeSeriesRepo will load the data when we need it, and remove it from memory when we're finished.  This allows for processing of massive data sets without running out of memory.

In [ ]:
outData2 = []
for point in series:
    newPoint = (point[0],point[1]*10)
    outData2.append(newPoint)

### Uploading data to SensorCloud

To upload data to SensorCloud we need to create a new data series.
For this example we'll upload the data to the same sensor with a modified channel name.
We also use the same samplerate of the existing channel.
If the sensor or channel doesn't yet exist on the device, they will be created when you upload data.

In [ ]:
outSeries = repo.createTimeSeries(inSensor, outChannel, series.getSampleRate(), series.getSampleRateType())

You can use the push() method to push a list of tuples into the data series

In [ ]:
outSeries.push(outData)

This data will be uploaded to SensorCloud when you call the save method

In [ ]:
outSeries.save()

# Tag the channel as a MathEngine channel so it will be deletable in the SensorCloud interface
outSeries.tagAsMathengine()

### Plot the data

In [ ]:
# Set the plot size and quality
matplotlib.rcParams['figure.figsize'] = (8,5)
matplotlib.rcParams['savefig.dpi'] = 100

In [ ]:
# X axis is 0 - n
arraySize = len(array)
xValues = np.arange(arraySize)

# Maximum value in the y axis
maxY = np.amax(array)*1.25

# Plot the values and fill in the space below them
plt.plot(xValues, array, color = 'k')
plt.fill_between(xValues, array, color = '#3779ca', alpha = 0.7)

# Set the bounds of the plot
# [xMin, xMax, yMin, yMax]
plt.axis([0, arraySize, 0, maxY])

# Turn on the grid
plt.grid(True)

# Title the plot and axis
plt.title( 'Sensor: ' + inSensor +  '\n' + 
           'Channel: ' + inChannel + '\n' +
           'Sampled at: ' + str(series.getSampleRate()) + ' ' + series.getSampleRateType())
plt.xlabel( 'X Axis' )
plt.ylabel( 'Y Axis' )